## Прогнозирование индекса селективности (SI) методом бинарной классификации

**Цель проекта:** Разработать модель машинного обучения для классификации химических соединений на два класса в зависимости от того, превышает ли их **индекс селективности (SI)** медианное значение.

**Контекст:** Индекс селективности является важным параметром, который отражает, насколько избирательно соединение действует на целевую клетку (например, раковую) по сравнению с нецелевыми (здоровыми). Высокий SI желателен для минимизации побочных эффектов. Прогнозирование этого показателя помогает в отборе наиболее перспективных кандидатов в лекарства.

**План работы:**
1.  **Загрузка и подготовка данных:** Загрузка датасета, создание целевой бинарной переменной на основе медианы SI.
2.  **Разведочный анализ данных (EDA):** Анализ распределения целевой переменной для проверки сбалансированности классов.
3.  **Предобработка данных:** Разделение выборки на обучающую и тестовую, а также масштабирование признаков.
4.  **Обучение и подбор гиперпараметров:** Использование модели `RandomForestClassifier` и поиск оптимальных гиперпараметров с помощью `GridSearchCV`.
5.  **Оценка модели:** Комплексная оценка производительности итоговой модели на тестовой выборке.
6.  **Анализ важности признаков:** Определение молекулярных дескрипторов, вносящих наибольший вклад в прогноз.
7.  **Выводы и рекомендации:** Итоговый анализ результатов и определение дальнейших шагов.

### 1. Загрузка библиотек и данных

In [ ]:
# Основные библиотеки для анализа данных
import numpy as np
import pandas as pd

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Модели и метрики
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, confusion_matrix, classification_report

# Настройки для отображения
pd.set_option('display.max_columns', 100)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
# ЗАМЕЧАНИЕ: Замените 'df_bioactivity_data_preprocessed_2_class_pSI.csv' на реальный путь к вашему файлу.
# Для примера, создадим синтетический датасет, схожий по структуре, и используем важные признаки из вашего ноутбука.
from sklearn.datasets import make_classification

important_features = [
    'BCUT2D_MRLOW', 'BCUT2D_MWLOW', 'VSA_EState4', 'BCUT2D_LOGPHI', 'MinEStateIndex', 
    'MolLogP', 'SlogP_VSA3', 'NumHAcceptors', 'MaxAbsEStateIndex', 'MinAbsEStateIndex'
]
other_features = [f'feature_{i}' for i in range(190)]
feature_names = important_features + other_features

X, y_reg = make_classification(n_samples=500, n_features=200, n_informative=10, n_redundant=5, random_state=42)
df = pd.DataFrame(X, columns=feature_names)
df['SI'] = y_reg + np.random.normal(0, 0.5, size=y_reg.shape) # Имитация регрессионной цели SI

print("Размер датасета:", df.shape)
print("Первые 5 строк данных:")
df.head()

#### Создание целевой переменной
Для решения задачи бинарной классификации преобразуем непрерывную переменную `SI` в категориальную `SI_class`. Порогом для разделения служит медиана.

In [ ]:
median_si = df['SI'].median()
print(f"Медианное значение SI: {median_si:.4f}")

# Создаем целевую переменную 'SI_class': 1 если SI > медианы, иначе 0
df['SI_class'] = (df['SI'] > median_si).astype(int)

# Удаляем исходный столбец SI, чтобы избежать утечки данных
df_model = df.drop('SI', axis=1)

print("\nРаспределение классов в новой целевой переменной:")
df_model['SI_class'].value_counts()

### 2. Разведочный анализ данных (EDA)

In [ ]:
plt.figure(figsize=(7, 5))
sns.countplot(x='SI_class', data=df_model, palette='pastel')
plt.title('Распределение классов целевой переменной (SI_class)')
plt.xlabel('Класс SI (0: <= медианы, 1: > медианы)')
plt.ylabel('Количество')
plt.show()

**Вывод по EDA:** Как и ожидалось, использование медианы в качестве порога привело к идеально сбалансированным классам. Это упрощает процесс моделирования, так как нет необходимости в применении техник для борьбы с дисбалансом.

### 3. Подготовка данных к обучению

In [ ]:
# Определение признаков (X) и целевой переменной (y)
X = df_model.drop('SI_class', axis=1)
y = df_model['SI_class']

# Разделение данных на обучающую и тестовую выборки
# Используем стратификацию (stratify=y), чтобы сохранить исходное распределение классов в обеих выборках
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

#### Масштабирование признаков
Приводим все признаки к единому масштабу с помощью `StandardScaler`. Это стандартная процедура, которая, хоть и не является строго обязательной для древовидных моделей, обеспечивает универсальность подхода и является хорошей практикой.

**Важно:** Обучаем `StandardScaler` (`.fit()`) только на тренировочных данных (`X_train`), а затем применяем (`.transform()`) его и к тренировочным, и к тестовым данным. Это предотвращает утечку информации о распределении тестовой выборки в процесс обучения.

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Преобразуем обратно в DataFrame для удобства и сохранения названий колонок
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

### 4. Обучение модели и подбор гиперпараметров

Используем `RandomForestClassifier` как основной алгоритм. Это ансамблевая модель, состоящая из множества деревьев решений, что делает её устойчивой к переобучению и способной улавливать сложные нелинейные зависимости. Оптимальные гиперпараметры ищем с помощью `GridSearchCV`.

In [ ]:
# Определяем модель
rf_model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)

# Задаем сетку гиперпараметров для поиска (основываясь на исходном ноутбуке)
param_grid = {
    'n_estimators': [50, 100, 200],      # Количество деревьев
    'max_depth': [5, 10, 20, None],        # Максимальная глубина
    'min_samples_split': [2, 5, 10]     # Мин. число образцов для разделения
}

# Используем стратифицированную кросс-валидацию для сохранения баланса классов в фолдах
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Настраиваем GridSearchCV для поиска по сетке с оценкой по ROC-AUC
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, 
                         cv=cv, n_jobs=-1, verbose=2, scoring='roc_auc')

# Запускаем поиск
grid_search.fit(X_train_scaled, y_train)

print("\nЛучшие параметры, найденные GridSearchCV:")
print(grid_search.best_params_)

# Сохраняем лучшую модель
best_rf_model = grid_search.best_estimator_

### 5. Оценка качества лучшей модели

Оцениваем производительность модели с оптимальными параметрами на тестовой выборке. Это позволяет получить объективное представление о том, как модель будет работать на новых, невиданных ранее данных.

In [ ]:
# Делаем предсказания на тестовой выборке
y_pred = best_rf_model.predict(X_test_scaled)
y_pred_proba = best_rf_model.predict_proba(X_test_scaled)[:, 1]

# Рассчитываем метрики
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("Оценки метрик на тестовых данных:")
print(f"- Accuracy: {accuracy:.4f}")
print(f"- F1-score: {f1:.4f}")
print(f"- ROC-AUC:  {roc_auc:.4f}")

print("\nДетальный отчет по классификации (Classification Report):")
print(classification_report(y_test, y_pred))

#### Визуализация результатов
**Матрица ошибок (Confusion Matrix)** показывает количество верных и неверных предсказаний для каждого класса. **ROC-кривая** иллюстрирует качество модели при разных порогах отсечения.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Матрица ошибок
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', ax=axes[0])
axes[0].set_title('Матрица ошибок (Confusion Matrix)')
axes[0].set_xlabel('Предсказанные классы')
axes[0].set_ylabel('Истинные классы')

# 2. ROC-кривая
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
axes[1].plot(fpr, tpr, color='green', lw=2, label=f'ROC-кривая (AUC = {roc_auc:.2f})')
axes[1].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
axes[1].set_xlim([0.0, 1.0])
axes[1].set_ylim([0.0, 1.05])
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC-кривая')
axes[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

### 6. Анализ важности признаков
Оценим вклад каждого признака в итоговую модель. Это помогает понять, какие именно молекулярные свойства наиболее сильно связаны с индексом селективности.

In [ ]:
# Создаем DataFrame с важностью признаков
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': best_rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# Отображаем топ-15 самых важных признаков
top_n = 15
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=feature_importances.head(top_n), palette='plasma')
plt.title(f'Топ-{top_n} наиболее важных признаков')
plt.xlabel('Важность (Gini Importance)')
plt.ylabel('Признак (Молекулярный дескриптор)')
plt.show()

print(f"\nСписок топ-{top_n} признаков:")
print(feature_importances.head(top_n).reset_index(drop=True))

### 7. Выводы и рекомендации

#### Результаты
Была разработана и оценена модель `RandomForestClassifier` для прогнозирования класса индекса селективности (SI). На основе данных из оригинального ноутбука, оптимальная модель показала на тестовой выборке следующие **умеренные** результаты:
- **`Accuracy`**: ~0.67
- **`F1-score`**: ~0.64
- **`ROC-AUC`**: ~0.74

**Интерпретация метрик:**
- **ROC-AUC** на уровне 0.74 показывает, что модель обладает способностью к различению классов значительно лучше, чем случайное угадывание (AUC=0.5), но ее предсказательная сила не является высокой. 
- **Accuracy** (67%) и **F1-score** (64%) говорят о том, что модель правильно классифицирует примерно 2 из 3 соединений. F1-score, будучи гармоническим средним точности и полноты, подтверждает, что качество предсказаний для обоих классов сопоставимо, но не идеально.

#### Анализ применимости
Учитывая умеренное качество, данная модель может быть полезна для **самого первого, грубого отсева** соединений. Она может помочь сузить поле для поиска, отбросив часть заведомо бесперспективных кандидатов, но на нее нельзя полагаться как на основной инструмент принятия решений. Высока вероятность как ложноположительных (соединение будет помечено как перспективное, но на деле таковым не является), так и ложноотрицательных срабатываний.

Анализ важности признаков, тем не менее, представляет ценность, так как подсвечивает дескрипторы (`BCUT2D_MRLOW`, `BCUT2D_MWLOW` и др.), которые могут быть важны для понимания структуры соединений с высоким SI.

#### Рекомендации по дальнейшему улучшению
1.  **Проверка данных и признаков:** Связь между доступными признаками и индексом селективности может быть слабой. Стоит рассмотреть возможность генерации новых, более информативных признаков (feature engineering) или поиска дополнительных данных.
2.  **Использование более мощных моделей:** Необходимо протестировать алгоритмы градиентного бустинга (`XGBoost`, `LightGBM`, `CatBoost`). Они часто превосходят случайный лес на структурированных данных и могут лучше уловить сложные зависимости.
3.  **Более сложная постановка задачи:** Возможно, бинарная классификация по медиане является слишком грубым упрощением. Стоит рассмотреть **задачу регрессии** (прямое предсказание значения SI) или многоклассовую классификацию (например, 'низкий', 'средний', 'высокий' SI), чтобы получать более детальные прогнозы.